# Extending Python with C (and C++)

Marco De Pascale . Py@AstroPD 30/01/2018

1. [Introduction](#introduction)
2. [Cython inside Jupiter](#cyjupiter)
3. [Cython outside Jypiter](#cynojupiter)
4. [Static typing](#faster)
5. [Examples](#examples)

I will introduce on the use on Cython as an easy way to extends Python with C/C++ code.  <a name="introduction"></a>

> _Cython is a superset of the Python programming language, designed to give **C**-like performance with code which is **mostly written in Python** [Wikipedia]_

[//]: <> (https://simplyml.com/python-ml/)

![](img/runtimes_202.png?raw=true)

> Cython works by producing a standard Python module. However, the behavior differs from standard Python in that the module code, originally written in Python, is translated into **C**

Just to stress it, as a standard Cython generates C code, **NOT C++**.

Cython is a fork of [Pyrex](http://www.cosc.canterbury.ac.nz/greg.ewing/python/Pyrex/), and is developed by, and for, [Sage](https://en.wikipedia.org/wiki/SageMath).

### Install

Now, did you install `cython`? 
Anaconda should ship with it, if not run:
```
conda install cython
```
Additionally, Cython needs the C compiler, so you should get it for you OS
+ **Linux**: get gcc from repository
+ **MacOS**: get Apple XCode from DVD or http://developer.apple.com 
+ **Windows**: use the open source MinDW. Cython documentation has an appendix on this (http://docs.cython.org/en/latest/src/tutorial/appendix.html).

---

## Cython inside Jupiter Notebook  <a name="cyjupiter"></a>

Use the magic symbol `%` !

In [1]:
# The following "magic" command is to enable support for Cython compilation
%load_ext Cython

In [23]:
# To compile the notebook cell, you need a second magic command %%cython.
# NOTE: It has to be the first instruction in the cell!

In [5]:
%%cython
#%%cython --annotate
from astropy import constants as const
def say_hello_to(name):
    print("Hello %s!" % name)
    print(const.c.value)

In [33]:
say_hello_to("World")

Hello World!
299792458.0


---

## Cython ouside Jupyter <a name="cynojupiter"></a>

This is the normal way of using Cython. You need to write 2 files:
+ the source code, with extension `.pyx`
+ a `setup.py` file to invoke the `setuptools` build process that generates the module

### The `source` code

Save following code in `hello.pyx`
```
def say_hello_to(name):
    print("Hello %s!" % name)
```

### The `setup.py` file

Save following code in `setup.py`

```
from distutils.core import setup
from Cython.Build import cythonize

setup(
  name = 'Hello world app',
  ext_modules = cythonize("hello.pyx"),
)
```

Build running the command
```
python setup.py build_ext --inplace
```

The above command generates the Cython source file(s)

Finally, from a Python session import the function to use it

In [28]:
from hello import say_hello_to
say_hello_to("Pippo")

Hello Pippo!


Quite long... but if you are not using the notebook, that is the way.
Probably you need this way to profile your code.

### Avoiding the `setup.py`

If you do not require extra C libraries, you can use the `pyximport` module. 
It will load `.pyx` files directly on import.

You would do something like:

In [29]:
import pyximport; pyximport.install()
from hello import say_hello_to

say_hello_to("pyximport")

Hello pyximport!


---

# Faster code via static typing <a name="faster"></a>

Cython is a Python compiler. This means that it can compile normal Python code without changes

> For performance critical code, it is often helpful to add **static type declarations**. They will allow Cython to generate simpler and **faster C code**.

This is achieved by stepping out of the dynamic nature of the Python code.

In [30]:
%%cython
cdef int i
cdef float f
# ...
cdef:
    int w, z
    bint b
    

All standard C types are available: `char`, `short`, `int`, `long`, `long long` as well as their `unsigned` version.

For booleans there is the special `bint` type.

You can declare also pointer as in C with the `*`: 
```
cdef int *n
```

And functions

```
cdef int eggs(unsigned long l, float f):
```

> This can make the source code more verbose and thus less readable. **It is therefore discouraged to use them without good reason**. Typically a few types in the right spots go a long way.

#### Right spots?!
Two essential tools to help with this task are [**profiling**](http://docs.cython.org/en/latest/src/tutorial/profiling_tutorial.html) and **annotation**....

---

## Examples <a name="examples"></a>

### Fibonacci

In [6]:
%%cython
#%%cython --annotate
def fib (n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        print(b),
        a, b = b, a + b

#### Adding some static typing (in rigth spots?)

In [7]:
%%cython
#%%cython --annotate
cdef fib (int n):
    """Print the Fibonacci series up to n."""
    #a, b = 0, 1
    cdef int a=0
    cdef int b=1
    while b < n:
        print(b),
        a, b = b, a + b